# Explore source measurement on a custom coadd

**Runs at:** data-int.lsst.cloud

<br>

**Goal:** For a custom i-band coadd that I have made with just a week's worth of inputs, run source detection, deblend, and measurement on the coadd. The contents of this NB will become part of Section 5 of "draft_create_custom_coadds.ipynb".

<br>

**Current Issue**

The `quanta = spe.run()` cell will execute, but the `lsst.deblend.multibandDeblend` step went on for >20 min, which is far too long, and made me realize we should just be doing single band deblending here.

From the <a href="https://pipelines.lsst.io/modules/lsst.pipe.tasks/tasks/lsst.pipe.tasks.multiBand.DeblendCoaddSourcesTask.html?highlight=multibanddeblend">docs page for multibandDeblend</a>, it looked like we could set the configuration parameter `simultaenous = False` in order to retarget to `singleBandDeblend`. So I did include this configuration override below:

> `measureSourcesPipeline.addConfigOverride('deblend', 'simultaneous', 'False')`

but now the `spe = SimplePipelineExecutor.from_pipeline(measureSourcesPipeline, where=queryString, butler=simpleButler)` cell yeilds the error message:

> `AttributeError: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiConfig has no attribute simultaneous`

So the question is, should we be setting our pipeline to just do single band deblending, and if so, how do we do that?

And maybe to step back -- should we even be doing the `mergeDetections` step if we only have a single filter?

<br> 

**Last run with:**

In [1]:
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

Weekly 2022_38
lsst_distrib          g0b29ad24fb+cd38383676 	current w_2022_38 setup


<br>

**Imports & Set Up**

In [2]:
import time
import numpy as np
import pandas

import lsst.geom
import lsst.afw.display as afwDisplay

from lsst.daf.butler import Butler, CollectionType

# lsst packages for executing pipeline tasks
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [3]:
config = "dp02"
collection = "u/melissagraham/coadd_recreation_nb"
outputRun = "u/melissagraham/coadd_recreation_nb/TestWindow1_sources"
my_dataId = {'band': 'i', 'tract': 4431, 'patch': 17}

<br>

**Optional:** Take a quick look at what I already have in the butler.

In [4]:
my_butler = Butler(config)

In [5]:
for c in sorted(my_butler.registry.queryCollections()):
    if c.find('melissagraham') > -1:
        print(c)

u/melissagraham/coadd_recreation_nb
u/melissagraham/coadd_recreation_nb/TestWindow1_coadd
u/melissagraham/coadd_recreation_nb/TestWindow1_sources


In [6]:
del my_butler

<br>

**Optional:** Delete `TestWindow1_sources` if it already exists, because we use it as the outputRun below.

In [7]:
my_butler = Butler(config, collections=[collection], writeable=True)

In [8]:
my_butler.removeRuns(['u/melissagraham/coadd_recreation_nb/TestWindow1_sources'])

In [9]:
for c in sorted(my_butler.registry.queryCollections()):
    if c.find('melissagraham') > -1:
        print(c)

u/melissagraham/coadd_recreation_nb
u/melissagraham/coadd_recreation_nb/TestWindow1_coadd


In [10]:
del my_butler

<br>

**Instantiate simpleButler**

In [11]:
simpleButler = Butler(config, collections=[collection], run=outputRun, writeable=True)

In [12]:
simpleButler.registry.getCollectionChain(collection)

CollectionSearch(('u/melissagraham/coadd_recreation_nb/TestWindow1_coadd', '2.2i/runs/DP0.2'))

**Check:** that the coadd being used is the 6-input coadd from `TestWindow1_coadd`.

In [13]:
my_coadd_inputs = simpleButler.get("deepCoadd.coaddInputs", my_dataId)
my_coadd_inputs.visits.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,goodpix,weight,filter
,pix,pix,pix,pix,,,
int64,int32,int32,int32,int32,int32,float64,str32
919515,11900,7900,16099,12099,8982709,3.4656688819793495,i_sim_1.4
924057,11900,7900,16099,12099,16098179,4.384267091685517,i_sim_1.4
924085,11900,7900,16099,12099,831332,4.446833161599578,i_sim_1.4
924086,11900,7900,16099,12099,16136708,4.550420295334223,i_sim_1.4
929477,11900,7900,16099,12099,16280498,4.051326013718346,i_sim_1.4
930353,11900,7900,16099,12099,16076133,3.7685753871220466,i_sim_1.4


<br>

**Set up the pipeline.**

In the DRP.yaml file, the source detection steps that come after assembleCoadd are:
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask
 - detection: lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask
 - mergeDetections: lsst.pipe.tasks.mergeDetections.MergeDetectionsTask
 - deblend: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiTask
 - measure: lsst.pipe.tasks.multiBand.MeasureMergedCoaddSourcesTask
 - mergeMeasurements: lsst.pipe.tasks.mergeMeasurements.MergeMeasurementsTask
 - writeObjectTable: lsst.pipe.tasks.postprocess.WriteObjectTableTask
 
Not needed are:
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask

Starting with `detection` and all the lists tasks after it.

**Which yaml file?**

Use `$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-DP0.2.yaml`, which has DP0-specific configurations, in particular the one we need to correctly set ref_cat for the measure task. This yaml file is now being used in draft_Create_Custom_Coadd as well.

In [14]:
yaml_file = '$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-DP0.2.yaml'
steps = 'detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable'
my_uri = yaml_file + '#' + steps
print(my_uri)

$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-DP0.2.yaml#detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable


In [15]:
measureSourcesPipeline = Pipeline.from_uri(my_uri)

<br>

**Create query string**

In [16]:
queryString = "tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'"
print(queryString)

tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'


<br>

**Configurations**

The first two were figured out by trial-and-error.

The third is needed because of not having the results from `finalizedCharacterization`.

In [21]:
measureSourcesPipeline.addConfigOverride('mergeDetections', 'priorityList', 'i')
measureSourcesPipeline.addConfigOverride('mergeMeasurements', 'priorityList', 'i')
measureSourcesPipeline.addConfigOverride('measure', 'propagateFlags.finalized_source_flags', {})

Try this in order to retarget for single-band deblending

In [22]:
measureSourcesPipeline.addConfigOverride('deblend', 'simultaneous', 'False')

<br>

**Optional** - Examine configs.

configs for measure -- why does the measure_config produce errors?

In [ ]:
# my_measure_config = simpleButler.get("measure_config", my_dataId)
# for key, value in my_measure_config.items():
#     print(key, value)

configs for deblend

In [26]:
# my_deblend_config = simpleButler.get("deblend_config", my_dataId)
# for key, value in my_deblend_config.items():
#     print(key, value)

configs for mergeMeasurements

In [24]:
# my_mergeMeasurements_config = simpleButler.get("mergeMeasurements_config", my_dataId)
# for key, value in my_mergeMeasurements_config.items():
#     print(key, value)

configs for writeObjectTable

In [ ]:
# my_writeObjectTable_config = simpleButler.get("writeObjectTable_config", my_dataId)
# for key, value in my_writeObjectTable_config.items():
#     print(key, value)

<br>

**Create SPE**

In [27]:
%%time
spe = SimplePipelineExecutor.from_pipeline(measureSourcesPipeline, where=queryString, butler=simpleButler)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/g6a99470703+d978885a28/python/lsst/pipe/tasks/multiBand.py:243: UserWarning: MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. Use MeasureMergedCoaddSourcesConfig.inputCatalog.
  warnings.warn("MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. "


AttributeError: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiConfig has no attribute simultaneous

**Run the pipeline.**

In [ ]:
%%time
quanta = spe.run()

In [ ]:
# del simpleButler

<br><br><br><br><br><br><br>


# earlier attempts. ignore below


## 2. Characterize Image Task

> **Help Question:** Is it necessary to run CharacterizeImageTask on the coadd? It seems the image already has psf, is already characterized?

In [ ]:
my_coadd_bbox = butler.get("deepCoadd.bbox", dataId=my_dataId)
x_val = my_coadd_bbox.beginX + 200
y_val = my_coadd_bbox.beginY + 200
point = lsst.geom.Point2D(x_val, y_val)

psf = my_coadd.getPsf()
psfShape = psf.computeShape(point)
sigma = psfShape.getDeterminantRadius()
pixelScale = my_coadd.getWcs().getPixelScale().asArcseconds()
print('psf fwhm = {:.2f} arcsec at x = {:.0f}, y = {:.0f}'.format(sigma*pixelScale*2.355,x_val,y_val))

If it's necessary, do like this?

In [ ]:
# from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

# ci_config = CharacterizeImageTask.ConfigClass()
# ci_config.psfIterations = 1
# charImageTask = CharacterizeImageTask(config=ci_config)

# result = charImageTask.run(my_coadd)

## 3. Configure Source Detection, Deblend, and Measurement Tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
tab = afwTable.SourceTable.make(schema)

In [ ]:
schema

In [ ]:
sd_config = SourceDetectionTask.ConfigClass()
sd_config.thresholdValue = 5
sd_config.thresholdType = "stdev"

sourceDetectionTask = SourceDetectionTask(schema=schema, config=sd_config)

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

In [ ]:
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
sm_config = SingleFrameMeasurementTask.ConfigClass()

sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=sm_config,
                                                   algMetadata=algMetadata)

## 4. Run Source Detection Task

In [ ]:
%%time
result = sourceDetectionTask.run(tab, my_coadd)

In [ ]:
sources = result.sources

In [ ]:
print(len(sources), result.numPosPeaks)

In [ ]:
sources.asAstropy()

In [ ]:
tx = np.where( np.isfinite( sources['coord_ra'] ) )[0]
print(len(tx))
del tx

## 5. Run Source Deblending

https://pipelines.lsst.io/modules/lsst.meas.deblender/index.html

https://pipelines.lsst.io/modules/lsst.meas.deblender/tasks/lsst.meas.deblender.SourceDeblendTask.html

> **Help Question:** `sourceDeblendTask.run` exists with an assertion error about the schemas even when we make sure the input schema matches before we start. Not sure how to fix.

SourceDeblendTask modifies the source catalog in-place.

In [ ]:
assert sources.getSchema() == schema

Attempt to fix that by redefining schema and sourceDeblendTask.

In [ ]:
schema = sources.getSchema()

In [ ]:
assert sources.getSchema() == schema

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

Does not seem to help, below we still get the error:
> `--> 275 assert sources.getSchema() == self.schema`

In [ ]:
%%time
sourceDeblendTask.run(my_coadd, sources)

In [ ]:
print(len(sources))

## 6. Run Source Measurement

https://pipelines.lsst.io/modules/lsst.meas.base/tasks/lsst.meas.base.sfm.SingleFrameMeasurementTask.html

In [ ]:
%%time
sourceMeasurementTask.run(measCat=sources, exposure=my_coadd)